In [26]:
def availableID(assignedIDs):
    pool=set(range(100))
    return list(pool-assignedIDs)[0]

assigned=set()
newID = availableID(assigned)
assigned.add(newID)
assigned, 

3

In [31]:
shortHands="""
node1
    b
    c:nodeD
    e:f g h
"""
def newPre(line): #IDs are provisional in preNodes
    preNode = dict( id = line.split(':')[0],
                    DEPTH = len(line) - len(line.lstrip()),
                    RAW = line,
                    PARENT = None)        
    return preNode

def getPreNodes(shortHands=shortHands):
    preNodes = [newPre(line) for line in shortHands.split('\n') if line]

    for i, preNode in enumerate(preNodes): #compute parents
        if i>1:
            for j in range(i+1):
                if preNodes[i-j]['DEPTH'] < preNode['DEPTH']: #found parent
                    preNode['PARENT'] = preNodes[i-j]  #parents
                    break # We now know our parent
    return preNodes
getPreNodes()

[{'id': 'node1', 'DEPTH': 0, 'RAW': 'node1', 'PARENT': None},
 {'id': '    b', 'DEPTH': 4, 'RAW': '    b', 'PARENT': None},
 {'id': '    c',
  'DEPTH': 4,
  'RAW': '    c:nodeD',
  'PARENT': {'id': 'node1', 'DEPTH': 0, 'RAW': 'node1', 'PARENT': None}},
 {'id': '    e',
  'DEPTH': 4,
  'RAW': '    e:f g h',
  'PARENT': {'id': 'node1', 'DEPTH': 0, 'RAW': 'node1', 'PARENT': None}}]

In [33]:
graphString = """

id  =simpleNode

id=node1
label=this is a label with an = sign 
title=multiline
title
thingy
URL = http://google.org

nodes=
color=red
shape=hexagon

edges=
color=RED

layout=
enabled=true

edges=
border=dotted
"""
def getRaw(longhand=graphString):
    lines = [line.strip() for line in graphString.split('\n\n') if line] #remove white matter
    kvs = [line.split('=',1) for line in lines] #list of [k,v] 
    stripped =  [[k.strip(), v.strip().split('\n')] for k,v in kvs] 
    nodeData =   [v  for k,v in stripped if k=='id']
    optData = [(k,v) for k,v in stripped if k!='id']
    return nodeData, optData
nodeData, optData = getRaw()
nodeData,optData

([['simpleNode'],
  ['node1',
   'label=this is a label with an = sign ',
   'title=multiline',
   'title',
   'thingy',
   'URL = http://google.org']],
 [('nodes', ['color=red', 'shape=hexagon']),
  ('edges', ['color=RED']),
  ('layout', ['enabled=true']),
  ('edges', ['border=dotted'])])

In [34]:
def getNodes(nodeData):
    nodes=[]
    for nodeDatum in nodeData:
        node=dict(id = nodeDatum[0])
        if len(nodeDatum)>1:
            remainder = nodeDatum[1:]
            for i,line in enumerate(remainder):
                if '=' in line:
                    prop, value = line.split('=',1) 
                    node[prop.strip()] = value
                else: #append the line to the previous value, as with multi-line labels 
                    node[prop]+='\n' + line

        nodes.append(node)
    return nodes
getNodes(nodeData)

[{'id': 'simpleNode'},
 {'id': 'node1',
  'label': 'this is a label with an = sign ',
  'title': 'multiline\ntitle\nthingy',
  'URL': ' http://google.org'}]

In [47]:
def toGraphString(nodes = getNodes(nodeData)):
    ret=[]
    for node in nodes:
        for k,v in node.items():
            ret.append(f'{k:6} = {v}')
        ret.append('\n')
    return '\n'.join(ret)
print(toGraphString()  )

id     = simpleNode


id     = node1
label  = this is a label with an = sign 
title  = multiline
title
thingy
URL    =  http://google.org




In [48]:
getNodes(getRaw(toGraphString())[0])

[{'id': 'simpleNode'},
 {'id': 'node1',
  'label': 'this is a label with an = sign ',
  'title': 'multiline\ntitle\nthingy',
  'URL': ' http://google.org'}]

In [55]:
oldGraphString = """
id     = simpleNode
linkto = node1

id     = node1
label  = this is a label with an = sign 
title  = multiline
title
thingy
URL    =  http://google.org
"""

newShortHands = """
simpleNode: b
node1
    node2
        node2b
"""

oldNodes =  getNodes(oldGraphString)

preNodes = getPreNodes(newShortHands)

nodeData, optData = getRaw(oldGraphString)
oldNodes = getNodes(nodeData)

preNodes, oldNodes

([{'id': 'simpleNode', 'DEPTH': 0, 'RAW': 'simpleNode: b', 'PARENT': None},
  {'id': 'node1', 'DEPTH': 0, 'RAW': 'node1', 'PARENT': None},
  {'id': '    node2',
   'DEPTH': 4,
   'RAW': '    node2',
   'PARENT': {'id': 'node1', 'DEPTH': 0, 'RAW': 'node1', 'PARENT': None}},
  {'id': '        node2b',
   'DEPTH': 8,
   'RAW': '        node2b',
   'PARENT': {'id': '    node2',
    'DEPTH': 4,
    'RAW': '    node2',
    'PARENT': {'id': 'node1', 'DEPTH': 0, 'RAW': 'node1', 'PARENT': None}}}],
 [{'id': 'simpleNode'},
  {'id': 'node1',
   'label': 'this is a label with an = sign ',
   'title': 'multiline\ntitle\nthingy',
   'URL': ' http://google.org'}])

In [ ]:
def splice(oldNodes = oldNodes, preNodes = freshNodes ):

In [5]:
def getOptions(optData=optData):
    options=dict(layout={}, nodes={}, edges={}, physics={})
    for optLine in optData:
        k = optLine[0] #nodes, edges, etc.
        for v in optLine[1]:  
            #print(v)
            prop, value = v.split('=',1)                #color , red
            prop, value = prop.strip(), value.strip()   #color,red
            if value == 'true': value=True
            if value == 'false': value=False
            options[k][prop] = value                    #options['nodes']['color'] = red

    for key in list(options.keys()): #remove e.g., options['physics']={} 
        if not options[key]:
            del options[key]
    return options        
getOptions(optData)

{'layout': {'enabled': True},
 'nodes': {'color': 'red', 'shape': 'hexagon'},
 'edges': {'color': 'RED', 'border': 'dotted'}}